### 一、机器学习超参数优化器介绍

- 基于网格搜索的超参数优化

&emsp;&emsp;目前来说sklearn中超参数优化器有四种，分别是GridSearchCV（网格搜索）、RandomizedSearchCV（随机网格搜索）、HalvingSearchCV（对半网格搜索）和HalvingRandomizedSearchCV（对半随机网格搜索）。其中网格搜索是通过枚举搜索出一组最优超参数，枚举的精度最高但效率最低，也就是网格搜索其实是精度最高的搜索算法，但往往伴随着巨大的计算量；而加入了随机网格搜索，则是随机选取了原始参数空间的子空间，然后在这个子空间内进行枚举，尽管还是枚举，但由于参数空间的缩小，计算量也会随之减少，并且伴随着这个参数子空间不断扩大（可人工修改参数），随机网格搜索的计算量和精度都将逼近网格搜索，简而言之随机网格搜索是一种牺牲精度换效率的搜索方式；相比随机网格搜索，对半网格搜索采用了类似锦标赛的筛选机制进行多轮的参数搜索，每一轮输入原始数据一部分数据进行模型训练，并且剔除一半的备选超参数。由于每一轮都只输入了一部分数据，因此不同备选超参数组的评估可能存在一定的误差，但由于每一轮都只剔除一半的超参数组而不是直接选出最优的超参数组，因此也拥有一定的容错性。不难发现，这个过程也像极了RFE过程——每一轮用一个精度不是最高的模型剔除一个最不重要的特征，即保证了执行效率、同时又保证了执行精度。

&emsp;&emsp;如果从一个宏观视角来看，随机网格搜索是通过减少备选参数组来减少计算量，而对半网格搜索则是减少带入的数据量，来减少计算量。二者其实都能一定程度提升超参数的搜索效率，但也存在损失精度的风险。当然，如果还想更进一步提高搜索效率，则可以考虑对半搜索和随机搜索的组合——对半随机网格搜索，这种搜索策略实际上就是对半搜索的思路+随机网格搜索的超参数空间，即在一个超参数子空间内进行多轮筛选，每一轮剔除一半的备选超参数组。这种方法的搜索效率是最高的，但同时精度也相对较差。

- 数据集准备

In [ ]:
# 基础数据科学运算库
import numpy as np
import pandas as pd

# 可视化库
import seaborn as sns
import matplotlib.pyplot as plt

# 时间模块
import time

import warnings
warnings.filterwarnings('ignore')

# sklearn库
# 数据预处理
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

# 实用函数
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

# 常用评估器
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# 网格搜索
from sklearn.model_selection import GridSearchCV

# 自定义评估器支持模块
from sklearn.base import BaseEstimator, TransformerMixin

# re模块相关
import inspect, re

# 其他模块
from tqdm import tqdm
import gc
import lightgbm as lgb

In [ ]:
# 读取数据
tcc = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# 标注连续/离散字段
# 离散字段
category_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
                'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
                'PaymentMethod']

# 连续字段
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
 
# 标签
target = 'Churn'

# ID列
ID_col = 'customerID'

# 验证是否划分能完全
assert len(category_cols) + len(numeric_cols) + 2 == tcc.shape[1]

# 连续字段转化
tcc['TotalCharges']= tcc['TotalCharges'].apply(lambda x: x if x!= ' ' else np.nan).astype(float)
tcc['MonthlyCharges'] = tcc['MonthlyCharges'].astype(float)

# 缺失值填补
tcc['TotalCharges'] = tcc['TotalCharges'].fillna(0)

# 标签值手动转化 
tcc['Churn'].replace(to_replace='Yes', value=1, inplace=True)
tcc['Churn'].replace(to_replace='No',  value=0, inplace=True)

In [ ]:
features = tcc.drop(columns=[ID_col, target]).copy()
labels = tcc['Churn'].copy()

&emsp;&emsp;同时，创建自然编码后的数据集以及经过时序特征衍生的数据集：

In [ ]:
# 划分训练集和测试集
train, test = train_test_split(tcc, random_state=22)

X_train = train.drop(columns=[ID_col, target]).copy()
X_test = test.drop(columns=[ID_col, target]).copy()

y_train = train['Churn'].copy()
y_test = test['Churn'].copy()

X_train_seq = pd.DataFrame()
X_test_seq = pd.DataFrame()

# 年份衍生
X_train_seq['tenure_year'] = ((72 - X_train['tenure']) // 12) + 2014
X_test_seq['tenure_year'] = ((72 - X_test['tenure']) // 12) + 2014

# 月份衍生
X_train_seq['tenure_month'] = (72 - X_train['tenure']) % 12 + 1
X_test_seq['tenure_month'] = (72 - X_test['tenure']) % 12 + 1

# 季度衍生
X_train_seq['tenure_quarter'] = ((X_train_seq['tenure_month']-1) // 3) + 1
X_test_seq['tenure_quarter'] = ((X_test_seq['tenure_month']-1) // 3) + 1

# 独热编码
enc = preprocessing.OneHotEncoder()
enc.fit(X_train_seq)

seq_new = list(X_train_seq.columns)

# 创建带有列名称的独热编码之后的df
X_train_seq = pd.DataFrame(enc.transform(X_train_seq).toarray(), 
                           columns = cate_colName(enc, seq_new, drop=None))

X_test_seq = pd.DataFrame(enc.transform(X_test_seq).toarray(), 
                          columns = cate_colName(enc, seq_new, drop=None))

# 调整index
X_train_seq.index = X_train.index
X_test_seq.index = X_test.index

In [ ]:
ord_enc = OrdinalEncoder()
ord_enc.fit(X_train[category_cols])

X_train_OE = pd.DataFrame(ord_enc.transform(X_train[category_cols]), columns=category_cols)
X_train_OE.index = X_train.index
X_train_OE = pd.concat([X_train_OE, X_train[numeric_cols]], axis=1)

X_test_OE = pd.DataFrame(ord_enc.transform(X_test[category_cols]), columns=category_cols)
X_test_OE.index = X_test.index
X_test_OE = pd.concat([X_test_OE, X_test[numeric_cols]], axis=1)

In [ ]:
X_train_OE = X_train_OE.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_test_OE = X_test_OE.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [ ]:
y_train

0       0
1       0
2       0
3       0
4       0
       ..
5277    0
5278    0
5279    1
5280    0
5281    0
Name: Churn, Length: 5282, dtype: int64

- 不进行超参数优化时模型表现

In [ ]:
gbm = lgb.LGBMClassifier()
gbm

LGBMClassifier()

In [ ]:
LGBMClassifier?

Init signature:
LGBMClassifier(
    boosting_type: str = 'gbdt',
    num_leaves: int = 31,
    max_depth: int = -1,
    learning_rate: float = 0.1,
    n_estimators: int = 100,
    subsample_for_bin: int = 200000,
    objective: Union[str, Callable, NoneType] = None,
    class_weight: Union[Dict, str, NoneType] = None,
    min_split_gain: float = 0.0,
    min_child_weight: float = 0.001,
    min_child_samples: int = 20,
    subsample: float = 1.0,
    subsample_freq: int = 0,
    colsample_bytree: float = 1.0,
    reg_alpha: float = 0.0,
    reg_lambda: float = 0.0,
    random_state: Union[int, numpy.random.mtrand.RandomState, NoneType] = None,
    n_jobs: int = -1,
    silent: Union[bool, str] = 'warn',
    importance_type: str = 'split',
    **kwargs,
)
Docstring:      LightGBM classifier.
Init docstring:
Construct a gradient boosting model.

Parameters
----------
boosting_type : str, optional (default='gbdt')
    'gbdt', traditional Gradient Boosting Decision Tree.
    'dart', Dropo

具体的模型训练过程和sklearn中其他模型一样，通过fit进行训练，并利用predict进行结果输出：

In [ ]:
# 训练模型
gbm.fit(X_train_OE, y_train)

LGBMClassifier()

然后输出预测结果，同样可以输出类别结果和概率预测结果：

In [ ]:
gbm.predict(X_test_OE)

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [ ]:
gbm.score(X_train_OE, y_train)

0.8905717531238168

In [ ]:
gbm.score(X_test_OE, y_test)

0.787052810902896

|准确率|训练集|测试集|
|:--:|:--:|:--:|
|LGBM原始模型|0.8905|0.7870|

- 确定优化参数

|Name|Description|      
|:--:|:--:| 
|num_leaves|一棵树上的叶子节点数，默认为 31| 
|max_depth|树的最大深度，默认值为 -1，表示无限制|
|min_split_gain|相当于min_impurity_decrease，再分裂所需最小增益。默认值为 0，表示无限制|
|min_child_weight|子节点的最小权重和。默认值为 1e-3。较大的 min_child_weight 可以防止过拟合| 
|min_child_samples|相当于min_samples_leaf，单个叶子节点上的最小样本数量。默认值为 20。较大的 min_child_samples 可以防止过拟合| 
|boosting_type| 使用的梯度提升算法类型，默认为GBDT|
|subsample_for_bin|该参数表示对连续变量进行分箱时（直方图优化过程）抽取样本的个数，默认取值为200000|
|learning_rate|学习率，即每次迭代中梯度提升的步长，默认值为 0.1| 
|n_estimators|迭代次数，即生成的基学习器的数量，默认值为 100|
|reg_alpha| L1 正则化系数，默认值为 0|
|reg_lambda| L2 正则化系数。默认值为 0|
|subsample|模型训练时抽取的样本数量，取值范围为 (0, 1]，表示抽样比例，默认为1.0| 
|subsample_freq|抽样频率，表示每隔几轮进行一次抽样，默认取值为0，表示不进行随机抽样|
|colsample_bytree|在每次迭代（树的构建）时，随机选择特征的比例，取值范围为 (0, 1]，默认为1.0|

|params|经验最优范围|
|:--:|:--:|
|num_leaves|range(20, 51, 2)|
|max_depth|range(5, 15, 2))| 
|learning_rate|np.linspace(0.01, 0.2, 5)|
|n_estimators|range(10, 160, 70)|
|boosting_type|['gbdt', 'goss']|
|colsample_bytree|[0.6, 0.8, 1.0]|
|reg_alpha|np.linspace(0.01, 0.1, 2)|
|reg_lambda|np.linspace(0.01, 0.1, 2)|

&emsp;&emsp;在确定了要调优哪些参数后，接下来就需要确定每个参数的搜索空间了，这一步也是直接关系到参数搜索效率的关键步骤。首先我们需要对参数搜索需要耗费的时间有基本的判断，才好进行进一步搜索策略的制定，否则极容易出现“仿佛永远等不到搜索停止”的情况出现。

### 三、基于网格搜索的超参数优化

#### 1.网格搜索注意事项

&emsp;&emsp;首先需要明确的是，参数空间内总备选参数组合的数量为各参数取值之积，且随着参数空间内每个参数取值增加而呈现指数级上升，且随着参数空间内参数维度增加（增加新的超参数）呈指数级上升，且二者呈现叠加效应。例如现有参数空间如下：

In [ ]:
# 参数空间有4个备选参数组合
parameter_space0 = {"min_samples_leaf": range(1, 3),
                    "min_samples_split": range(1, 3)
                   }

则备选的参数组合有$2*2=4$个。而此时如果调整"min_samples_leaf": range(1, 4)，则备选参数组合就变成了$2*3=6$个，也就是说,"min_samples_leaf"参数搜索范围增加1，造成的搜索次数增加了两次，而非一次。

In [ ]:
# 参数空间有6个备选参数组合
parameter_space1 = {"min_samples_leaf": range(1, 4),
                    "min_samples_split": range(1, 3)
                   }

并且，如果我们新增一个超参数维度"max_depth": range(1, 4)，则目前总共的备选参数组合就达到了$2*3*3=18$个，也就是说,增加"min_samples_split"3个数值，造成的搜索次数增加了18-6=12次，而非3次：

In [ ]:
# 参数空间有18个备选参数组合
parameter_space2 = {"min_samples_leaf": range(1, 4),
                    "min_samples_split": range(1, 3), 
                    "max_depth": range(1, 4)
                   }

&emsp;&emsp;当然，这种指数级的变化在少量数据情况下可能无法看出“真正的威力”，但如果参数稍微多些或计算过程稍微复杂些，例如假设parameter_space1搜索任务耗时5分钟，而在只增加了一个参数及3个不同取值的情况下，parameter_space2就将耗费15分钟。而如果更复杂些，不是5\*3=15分钟，而是15\*3=45分钟呢，甚至是1小时\*3=3小时呢，参数空间的略微扩大就可能造成搜索时间的指数级增加。

> 此外，在进行网格搜索时，每一次建模背后还存在5折交叉验证，也就是需要训练5次模型，而每一次建模，都伴随着几十个甚至是上百个决策树模型训练，背后的计算量可想而知。

&emsp;&emsp;介于此，在参数空间设计时就会有这样一个核心问题，那就是参数空间设置小了不确定最优参数是否在这个空间内，参数空间设置大了又不确定何时能算完。这也就是所谓的参数空间设计时面临的“舍罕王赏麦”问题。

#### 2.超参数搜索的“凸函数”假设

&emsp;&emsp;如何解决这个问题，最好的解决方案是“小步迭代、快速调整”。在介绍这种方案之前，要先介绍在超参数调优时大家都会默认的一个假设，那就是超参数的取值和模型效果往往呈现严格“凸函数”的特性，例如假设参数"min_samples_leaf"在取值为5时模型效果最好，那么在参数取值为1、2、3、4时，模型效果是依次递增的，而如果参数取值为6、7、8，则模型效果是依次递减的，因此如果我们设计的该参数的搜索空间是"min_samples_leaf": range(6, 9)，参数在6、7、8之间取值，则最优结果将会是min_samples_leaf=6，即预设的参数空间的下届，此时我们就需要进一步的移动参数空间，例如改为"min_samples_leaf": range(5, 8)，即让参数在5、6、7之间取值，很明显，最终输出的挑选结果将会是min_samples_leaf=5，但此时仍然是搜索空间的下届，因此我们还需要进一步移动搜索空间，即移动至"min_samples_leaf": range(4, 7)，即让参数在4、5、6之间取值，此时输出的最优结果将会是min_samples_leaf=5，此时就无需再移动超参数空间了，因为此时的参数空间已经包括了“凸函数”的最小值点，再往左边移动没有任何意义，这个过程如下图所示：

<center><img src="https://s2.loli.net/2022/05/01/R5gubzxKyrVeWcd.png" alt="image-20220501213631356" style="zoom:50%;" />

对于单个参数来说，如果呈现出搜索空间包含了最优值点（或者最优值点不在搜索空间的边界上）时，则判断已经找到了最优超参数。

> 如果超参数的取值不仅是数值，而是数值和其他类型对象混合的情况，则其他类型对象需要单独作为一个备选项参与搜索。

&emsp;&emsp;对于单个变量是如此，对于多个变量来说也是如此，若最终超参数搜索结果呈以下状态，则说明我们已经找到了一组最优超参数组：

<center><img src="https://s2.loli.net/2022/05/01/vWeEnyfFau6soO8.png" alt="image-20220501214514625" style="zoom:50%;" />

> 当然，这种“凸函数假设”其实并没有充份严谨的理论依据，更多的是人们长期实践总结出来的结论。

#### 3.小步前进，快速调整

&emsp;&emsp;接下来我们来看如何通过“小步迭代快速调整”的方法来进行超参数的搜索。在这个策略里，我们每次需要设置一个相对较小的参数搜索空间，然后快速执行一次超参数搜索，并根据超参数搜索结果来调整参数空间，并进行更进一步的超参数搜索，如此往复，直到参数空间内包含了全部参数的最优解为止。就像此前举例的那样，我们不会给"min_samples_leaf"一次设置一个非常大的参数搜索范围（如[1,9]），而是每次设置一个更小的搜索范围，通过不断调整这个范围来定位最优解。

&emsp;&emsp;既然要反复执行搜索任务，就必然需要一定程度控制单次搜索任务所需要的时间。当然，单次搜索的时间会和CPU、数据量、参数空间大小有关，但一般来说，对于小样本，单次搜索任务最好控制在5-30min内，而对于海量样本，最好也控制在30min-2H内，特殊情况可以适当放宽单次搜索任务的时间。

&emsp;&emsp;不过无论单次搜索任务耗时或长或短，我们都需要首先有个大概的预判，即本次搜索需要多久，方便我们确定“下次回来看结果”的时间。这里我们以Telco原始数据集为例，来简单测试单次搜索任务需要的时间。这里我们先测试最短单次搜索需要耗费的时间，由于我们需要让每个最优参数落在某个区间的中间，因此每个超参数的取值范围区间至少包含三个数值，例如"min_samples_leaf": range(4, 7)、该参数本次搜索至少有三个备选值，此外，如果有些参数包含非数值型参数，则需要在数值参数区间基础上再加上一个非数值型参数，例如"max_samples":\[None, 0.6, 0.5, 0.4\]。

#### 4.首次搜索时超参数选取及取值范围的经验依据

|params|经验最优范围|
|:--:|:--:|
|num_leaves|range(20, 51, 2)|
|max_depth|range(5, 15, 2))| 
|learning_rate|np.linspace(0.01, 0.2, 5)|
|n_estimators|range(10, 160, 70)|
|boosting_type|['gbdt', 'goss']|
|colsample_bytree|[0.6, 0.8, 1.0]|
|reg_alpha|np.linspace(0.01, 0.1, 2)|
|reg_lambda|np.linspace(0.01, 0.1, 2)|

&emsp;&emsp;在设置了初始参数后，接下来就是一轮轮搜索与调整了，我们需要大致掌握每一次搜索任务所需要耗费的时间，然后在每次搜索任务结束时及时回到电脑前，准备设置调整参数空间并进行下一次搜索。

#### 5.分批训练策略

&emsp;&emsp;并且需要注意的是，在进行超参数搜索时，超参数彼此之间是存在交叉影响的，因此如果某次搜索只带入了部分参数进行搜索，那么如果后续增加了其他参数，则再次搜索时这些超参数的最优值也会发生变化。例如某次搜索超参数A在[1,2,3]中取值，找到了最优值A=2，现在如果继续加入超参数B，同时搜索A在[1,2,3]和B在[2,3,4]中最优取值组合，则极有可能出现A的最优取值变成了A=3，此时就要移动A的取值范围了（最优值落在了边界上），接下来如果继续加入超参数C、超参数D、超参数E等，每次加入一个都需要重新搜索一次，这个过程就会变得非常麻烦。当然，需要注意的是，如果只有A和B两个超参数，那么确实可以先搜索A、再搜索B，因为在两个超参数的情况下，二者相互影响有限，单独围绕A搜索出来的最优值2，在加入超参数B之后，A的最优值极有可能仍然在2附近变动，此时我们可以以2为中心设置搜索范围，之前搜索出来的A=2的最优值结果，在同时搜索A和B时仍然具有参考价值。但如果后续加入了C、D、E等更多的超参数，由于超参数彼此之间相互影响也会呈现指数级变动，因此极有可能后续A的取值会偏离2较远，有可能会变成10、20甚至是30，此时反观最开始搜索出来的A=2的最优值，对后续A的搜索过程就变得毫无价值了。

&emsp;&emsp;因此，受此启发，一般来说如果超参数个数较多，则可以分两批、甚至是分三批进行搜索，例如有A、B、C、D、E五个超参数时，可以先搜索A、B、C，在搜索出一组最优值后，再以此为中心创建搜索空间并加入新的D、E两个参数，设置各自对应的搜索空间，并进行第二批搜索。基本过程如下：

<center><img src="https://s2.loli.net/2022/05/02/Utqh532l7dfnJSj.png" alt="image-20220502165350402" style="zoom:50%;" />

> 总之，最终一定要得到一个全部超参数每个最优点都在给定区间范围内的结果。

|params|经验最优范围|
|:--:|:--:|
|num_leaves|range(20, 51, 2)|
|max_depth|range(5, 15, 2))| 
|learning_rate|np.linspace(0.01, 0.2, 5)|
|n_estimators|range(10, 160, 70)|
|boosting_type|['gbdt', 'goss']|
|colsample_bytree|[0.6, 0.8, 1.0]|
|(第一阶段)reg_alpha|np.linspace(0.01, 0.1, 2)|
|(第二阶段)reg_lambda|np.linspace(0.01, 0.1, 2)|

#### 6.LGBM网格搜索调参实战

&emsp;&emsp;在有了网格搜索优化技巧的基础知识储备后，接下来我们围绕Telco原生数据集来进行随机森林网格搜索实战。一方面测试在原始数据集情况下随机森林模型超参数优化的最好结果，同时我们也将用过一个实例来具体观察我们制定的“小步迭代、快速调整”的调优策略是否能真的帮助我们高效快速的确定最优超参数。

- 首轮搜索

&emsp;&emsp;首先，根据此前介绍，设置初始参数空间并进行搜索，同时计算本次运行的时间。原始数据集总共有19条特征，开方运算与log2计算结果如下：

In [ ]:
list(np.linspace(0.01, 0.2, 5))

[0.01, 0.0575, 0.105, 0.15250000000000002, 0.2]

In [ ]:
start = time.time()

# 设置超参数空间
parameter_space = {
    "num_leaves": range(20, 51, 2), 
    "max_depth": range(5, 15, 2),
    "learning_rate": list(np.linspace(0.01, 0.2, 5)),
    "n_estimators": range(10, 160, 70), 
    "boosting_type":['gbdt', 'goss'], 
    "colsample_bytree":[0.6, 0.8, 1.0]}

# 实例化模型与评估器
LGBM_0 = LGBMClassifier(random_state=12)
grid_LGBM_0 = GridSearchCV(LGBM_0, parameter_space, n_jobs=15)

# 模型训练
grid_LGBM_0.fit(X_train_OE, y_train)

print(time.time()-start)

124.77974891662598


&emsp;&emsp;然后查看当前情况下模型预测结果：

In [ ]:
grid_LGBM_0.best_score_

0.8078368237722543

In [ ]:
grid_LGBM_0.score(X_train_OE, y_train), grid_LGBM_0.score(X_test_OE, y_test)

(0.8528966300643696, 0.7864849517319704)

能够看出，在进行第一轮超参数搜索时，模型结果的过拟合倾向已经得到了有效抑制，并且对比此前逻辑回归最终的优化结果，目前模型已经得到了一个较好的结果了：

|Models|CV.best_score_|train_score|test_score|
|:--:|:--:|:--:|:--:|
|Logistic+grid|0.8045|0.8055|0.7932|
|RF+grid_R1|0.8084|0.8517|0.7848|

&emsp;&emsp;最后，重点关注本轮搜索得出的超参数最优取值：

In [ ]:
grid_LGBM_0.best_params_

{'boosting_type': 'gbdt',
 'colsample_bytree': 0.6,
 'learning_rate': 0.0575,
 'max_depth': 11,
 'n_estimators': 80,
 'num_leaves': 32}

- 第二轮搜索

In [ ]:
start = time.time()

# 设置超参数空间
parameter_space = {
    "num_leaves": range(30, 35), 
    "max_depth": range(10, 14),
    "learning_rate": list(np.linspace(0.01, 0.1, 5)),
    "n_estimators": range(70, 91, 5), 
    "boosting_type":['gbdt', 'goss'], 
    "colsample_bytree":[0.5, 0.6, 0.7]}

# 实例化模型与评估器
LGBM_1 = LGBMClassifier(random_state=12)
grid_LGBM_1 = GridSearchCV(LGBM_1, parameter_space, n_jobs=15)

# 模型训练
grid_LGBM_1.fit(X_train_OE, y_train)

print(time.time()-start)

54.85588598251343


In [ ]:
grid_LGBM_1.best_score_

0.8091622229867264

In [ ]:
grid_LGBM_1.score(X_train_OE, y_train), grid_LGBM_1.score(X_test_OE, y_test)

(0.8511927300265051, 0.7864849517319704)

In [ ]:
grid_LGBM_1.best_params_

{'boosting_type': 'gbdt',
 'colsample_bytree': 0.6,
 'learning_rate': 0.05500000000000001,
 'max_depth': 13,
 'n_estimators': 70,
 'num_leaves': 33}

- 第三轮搜索

In [ ]:
start = time.time()

# 设置超参数空间
parameter_space = {
    "num_leaves": range(30, 35), 
    "max_depth": range(12, 17),
    "learning_rate": list(np.linspace(0.04, 0.07, 5)),
    "n_estimators": range(60, 81, 2), 
    "boosting_type":['gbdt', 'goss'], 
    "colsample_bytree":[0.55, 0.6, 0.65]}

# 实例化模型与评估器
LGBM_2 = LGBMClassifier(random_state=12)
grid_LGBM_2 = GridSearchCV(LGBM_2, parameter_space, n_jobs=15)

# 模型训练
grid_LGBM_2.fit(X_train_OE, y_train)

print(time.time()-start)

137.77449536323547


In [ ]:
grid_LGBM_2.best_score_

0.810297690719876

In [ ]:
grid_LGBM_2.score(X_train_OE, y_train), grid_LGBM_2.score(X_test_OE, y_test)

(0.8487315410829231, 0.787052810902896)

In [ ]:
grid_LGBM_2.best_params_

{'boosting_type': 'gbdt',
 'colsample_bytree': 0.6,
 'learning_rate': 0.05500000000000001,
 'max_depth': 15,
 'n_estimators': 66,
 'num_leaves': 33}

- 第四轮搜索

In [ ]:
start = time.time()

# 设置超参数空间
parameter_space = {
    "num_leaves": range(32, 37), 
    "max_depth": range(14, 18),
    "learning_rate": list(np.linspace(0.05, 0.06, 5)),
    "n_estimators": range(65, 70), 
    "boosting_type":['gbdt', 'goss'], 
    "colsample_bytree":[0.59, 0.6, 0.61]}

# 实例化模型与评估器
LGBM_3 = LGBMClassifier(random_state=12)
grid_LGBM_3 = GridSearchCV(LGBM_3, parameter_space, n_jobs=15)

# 模型训练
grid_LGBM_3.fit(X_train_OE, y_train)

print(time.time()-start)

49.448071002960205


In [ ]:
grid_LGBM_3.best_score_

0.810297690719876

In [ ]:
grid_LGBM_3.score(X_train_OE, y_train), grid_LGBM_3.score(X_test_OE, y_test)

(0.8487315410829231, 0.787052810902896)

In [ ]:
grid_LGBM_3.best_params_

{'boosting_type': 'gbdt',
 'colsample_bytree': 0.59,
 'learning_rate': 0.055,
 'max_depth': 15,
 'n_estimators': 66,
 'num_leaves': 33}

- 第五轮搜索

In [ ]:
start = time.time()

# 设置超参数空间
parameter_space = {
    "num_leaves": range(32, 37), 
    "max_depth": range(14, 18),
    "learning_rate": list(np.linspace(0.05, 0.06, 5)),
    "n_estimators": range(65, 70), 
    "boosting_type":['gbdt', 'goss'], 
    "colsample_bytree":[0.585, 0.59, 0.595]}

# 实例化模型与评估器
LGBM_4 = LGBMClassifier(random_state=12)
grid_LGBM_4 = GridSearchCV(LGBM_4, parameter_space, n_jobs=15)

# 模型训练
grid_LGBM_4.fit(X_train_OE, y_train)

print(time.time()-start)

51.1261146068573


In [ ]:
grid_LGBM_4.best_score_

0.810297690719876

In [ ]:
grid_LGBM_4.score(X_train_OE, y_train), grid_LGBM_4.score(X_test_OE, y_test)

(0.8487315410829231, 0.787052810902896)

In [ ]:
grid_LGBM_4.best_params_

{'boosting_type': 'gbdt',
 'colsample_bytree': 0.585,
 'learning_rate': 0.055,
 'max_depth': 15,
 'n_estimators': 66,
 'num_leaves': 33}

- 最终轮搜索

In [ ]:
start = time.time()

# 设置超参数空间
parameter_space = {
    "num_leaves": range(32, 37), 
    "max_depth": range(14, 18),
    "learning_rate": list(np.linspace(0.05, 0.06, 5)),
    "n_estimators": range(65, 70), 
    "boosting_type":['gbdt', 'goss'], 
    "colsample_bytree":[0.585, 0.59, 0.595], 
    "reg_alpha":list(np.linspace(0.01, 0.1, 2))}

# 实例化模型与评估器
LGBM_5 = LGBMClassifier(random_state=12)
grid_LGBM_5 = GridSearchCV(LGBM_5, parameter_space, n_jobs=15)

# 模型训练
grid_LGBM_5.fit(X_train_OE, y_train)

print(time.time()-start)

104.39109444618225


In [ ]:
grid_LGBM_5.best_score_

0.8084039305065795

In [ ]:
grid_LGBM_5.score(X_train_OE, y_train), grid_LGBM_5.score(X_test_OE, y_test)

(0.84967815221507, 0.7904599659284497)

In [ ]:
grid_LGBM_5.best_params_

{'boosting_type': 'gbdt',
 'colsample_bytree': 0.585,
 'learning_rate': 0.052500000000000005,
 'max_depth': 14,
 'n_estimators': 69,
 'num_leaves': 36,
 'reg_alpha': 0.01}

|准确率|训练集|测试集|
|:--:|:--:|:--:|
|LGBM原始模型|0.8905|0.7870|
|LGBM+TPE+100|0.8197|0.7864|
|LGBM+TPE+200|0.8089|0.7870|
|LGBM+TPE+1000|0.80144|0.7890|
|LGBM+grid|0.8496|0.7904|

### 四、通过交叉训练进一步提升模型效果

<center><img src="https://s2.loli.net/2022/05/18/1UAlpgwt2mSbTQ5.png" alt="image-20220518205529331" style="zoom:33%;" />

In [ ]:
kf = KFold(n_splits=5, random_state=12, shuffle=True)

In [ ]:
# 实例化KFold评估器
kf = KFold(n_splits=5, random_state=12, shuffle=True)

# 重置训练集和测试集的index
X_train_OE = X_train_OE.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

然后需要单独创建每一轮划分后的训练集和验证集，可以通过如下过程完成：

In [ ]:
# 循环一次，切分一次数据集和验证集
for train_part_index, eval_index in kf.split(X_train_OE, y_train):
    print(train_part_index)
    print(eval_index)
    break

[   1    2    5 ... 5279 5280 5281]
[   0    3    4 ... 5271 5274 5275]


In [ ]:
X_train_OE.loc[train_part_index]

,gender,SeniorCitizen,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,tenure,MonthlyCharges,TotalCharges
1,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0,2.0,3,80.00,241.30
2,1.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,3.0,4,19.00,73.45
5,0.0,0.0,1.0,1.0,1.0,2.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,69,84.45,5848.60
6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,26,54.75,1406.90
7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,11,44.65,472.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5277,1.0,0.0,1.0,0.0,1.0,0.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,52,106.30,5487.00
5278,0.0,1.0,0.0,0.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,16,54.10,889.00
5279,0.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,2.0,2.0,0.0,2.0,2.0,0.0,1.0,2.0,28,106.15,3152.50
5280,0.0,0.0,1.0,1.0,1.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,15,20.35,335.95


In [ ]:
train_part_index_l = []
eval_index_l = []

for train_part_index, eval_index in kf.split(X_train_OE, y_train):
    train_part_index_l.append(train_part_index)
    eval_index_l.append(eval_index)

In [ ]:
train_part_index_l

[array([   1,    2,    5, ..., 5279, 5280, 5281]),
 array([   0,    1,    2, ..., 5279, 5280, 5281]),
 array([   0,    1,    2, ..., 5279, 5280, 5281]),
 array([   0,    1,    3, ..., 5275, 5277, 5281]),
 array([   0,    2,    3, ..., 5278, 5279, 5280])]

- 基于clf的交叉训练

In [ ]:
test_predict = []
kf = KFold(n_splits=3, random_state=11, shuffle=True)

for train_part_index, eval_index in kf.split(X_train_OE, y_train):
    # 在训练集上训练
    X_train_part = X_train_OE.loc[train_part_index]
    y_train_part = y_train.loc[train_part_index]
    clf.fit(X_train_part, y_train_part)
    # 将测试集上预测结果填入predict数据集
    test_predict.append(clf.predict_proba(X_test_OE)[:, 1])

In [ ]:
test_predict

[array([0.0307229 , 0.22867404, 0.02423184, ..., 0.11940864, 0.62738151,
        0.1207684 ]),
 array([0.02976366, 0.34260337, 0.01902775, ..., 0.17791131, 0.61817475,
        0.14663852]),
 array([0.03098913, 0.30166054, 0.01886396, ..., 0.19259972, 0.66194877,
        0.13475334])]

In [ ]:
np.array(test_predict).mean(0)

array([0.0304919 , 0.29097931, 0.02070785, ..., 0.16330656, 0.63583501,
       0.13405342])

In [ ]:
res = (np.array(test_predict).mean(0) >= 0.5) * 1

In [ ]:
res

array([0, 0, 0, ..., 0, 1, 0])

In [ ]:
y_test

0       0
1       1
2       0
3       0
4       0
       ..
1756    0
1757    0
1758    0
1759    1
1760    0
Name: Churn, Length: 1761, dtype: int64

In [ ]:
accuracy_score(res, y_test)

0.7910278250993753

|准确率|训练集|测试集|
|:--:|:--:|:--:|
|LGBM原始模型|0.8905|0.7870|
|LGBM+TPE+100|0.8197|0.7864|
|LGBM+TPE+200|0.8089|0.7870|
|LGBM+TPE+1000|0.80144|0.7890|
|LGBM+grid|0.8496|0.7904|
|LGBM+TPE+1000|交叉训练|0.7910|

- 基于grid_LGBM_5的交叉训练

In [ ]:
test_predict = []
kf = KFold(n_splits=3, random_state=11, shuffle=True)

for train_part_index, eval_index in kf.split(X_train_OE, y_train):
    # 在训练集上训练
    X_train_part = X_train_OE.loc[train_part_index]
    y_train_part = y_train.loc[train_part_index]
    grid_LGBM_5.best_estimator_.fit(X_train_part, y_train_part)
    # 将测试集上预测结果填入predict数据集
    test_predict.append(grid_LGBM_5.predict_proba(X_test_OE)[:, 1])

In [ ]:
test_predict

[array([0.02279709, 0.19923274, 0.02415225, ..., 0.10446081, 0.65781821,
        0.06689876]),
 array([0.0228167 , 0.37510663, 0.01549221, ..., 0.20079375, 0.62692221,
        0.10498177]),
 array([0.02746028, 0.33716952, 0.01307974, ..., 0.2329891 , 0.65084317,
        0.08372255])]

In [ ]:
np.array(test_predict).mean(0)

array([0.02435802, 0.30383629, 0.01757473, ..., 0.17941455, 0.64519453,
       0.08520103])

In [ ]:
res = (np.array(test_predict).mean(0) >= 0.5) * 1

In [ ]:
y_test

0       0
1       1
2       0
3       0
4       0
       ..
1756    0
1757    0
1758    0
1759    1
1760    0
Name: Churn, Length: 1761, dtype: int64

In [ ]:
accuracy_score(res, y_test)

0.7921635434412265

|准确率|训练集|测试集|
|:--:|:--:|:--:|
|LGBM原始模型|0.8905|0.7870|
|LGBM+TPE+100|0.8197|0.7864|
|LGBM+TPE+200|0.8089|0.7870|
|LGBM+TPE+1000|0.80144|0.7890|
|LGBM+grid|0.8496|0.7904|
|LGBM+TPE+1000|交叉训练|0.7910|
|LGBM+grid|交叉训练|0.7921|